# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

import matplotlib.pyplot as plt

from torch import cuda

from ate_models.ATE_BERT_Dropout_Linear import ATE_BERT_Dropout_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ATE_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,iob_aspect_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8
SEQ_LEN = 512

NO_RUNS = 10

In [10]:
BERT_FINE_TUNED_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_512.pth'

In [11]:
MODEL_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_linear_512.pth'
STATS_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_linear_512.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    losses = []

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))

        losses.append(loss.item())
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()
    
    return losses

In [16]:
def validation(model, dataloader, loss_fn):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    losses = []

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            losses.append(loss_fn(outputs.view(-1, 3), tags_tensors.view(-1)).item())

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets, losses

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ATE_BERT_Dropout_Linear(BertModel.from_pretrained('bert-base-uncased'), dropout=0.3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    train_losses = []
    for epoch in range(EPOCHS):
        losses = train(epoch, model, loss_fn, optimizer, train_dataloader)

        train_losses += losses
    
    plt.title(f'Train Loss for run {i + 1}/{NO_RUNS}')
    plt.plot(train_losses)
    plt.savefig(f'../../../results/ATE/SemEval16 - Task 5 - Restaurants/plots/bert_pt_dropout_linear_train_loss_run_{i + 1}.png')

    plt.clf()

    outputs, targets, valid_losses = validation(model, validation_dataloader, torch.nn.CrossEntropyLoss())
    plt.title(f'Valid Loss for run {i + 1}/{NO_RUNS}')
    plt.plot(valid_losses)
    plt.savefig(f'../../../results/ATE/SemEval16 - Task 5 - Restaurants/plots/bert_pt_dropout_linear_valid_loss_run_{i + 1}.png')
    plt.clf()
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, BERT_FINE_TUNED_OUTPUT)
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.192570686340332
Epoch: 0/2, Batch: 50/501, Loss: 0.042454976588487625
Epoch: 0/2, Batch: 100/501, Loss: 0.04388090968132019
Epoch: 0/2, Batch: 150/501, Loss: 0.024088304489850998
Epoch: 0/2, Batch: 200/501, Loss: 0.01759125106036663
Epoch: 0/2, Batch: 250/501, Loss: 0.016931114718317986
Epoch: 0/2, Batch: 300/501, Loss: 0.014734683558344841
Epoch: 0/2, Batch: 350/501, Loss: 0.0239996537566185
Epoch: 0/2, Batch: 400/501, Loss: 0.026723774150013924
Epoch: 0/2, Batch: 450/501, Loss: 0.010271296836435795
Epoch: 0/2, Batch: 500/501, Loss: 0.015817493200302124
Epoch: 1/2, Batch: 0/501, Loss: 0.010078986175358295
Epoch: 1/2, Batch: 50/501, Loss: 0.020399663597345352
Epoch: 1/2, Batch: 100/501, Loss: 0.004418396390974522
Epoch: 1/2, Batch: 150/501, Loss: 0.0047532920725643635
Epoch: 1/2, Batch: 200/501, Loss: 0.008206851780414581
Epoch: 1/2, Batch: 250/501, Loss: 0.0044282954186201096
Epoch: 1/2, Batch: 300/501, Loss: 0.009738890454173088
Epoch: 1/2, 

<Figure size 432x288 with 0 Axes>

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.997281,0.997281,0.769768,0.997281,0.646236,0.997281,0.668023,333.847623
1,0.997258,0.997258,0.752266,0.997258,0.704952,0.997258,0.710621,332.535915
2,0.996762,0.996762,0.759255,0.996762,0.620001,0.996762,0.608690,332.555571
3,0.997117,0.997117,0.741001,0.997117,0.707137,0.997117,0.718545,332.509994
4,0.997305,0.997305,0.763772,0.997305,0.679450,0.997305,0.680584,332.306395
5,0.997219,0.997219,0.723549,0.997219,0.752048,0.997219,0.733329,332.545011
6,0.997266,0.997266,0.784276,0.997266,0.700438,0.997266,0.702756,331.933245
7,0.997234,0.997234,0.738814,0.997234,0.730842,0.997234,0.725397,331.677276
8,0.997871,0.997871,0.799493,0.997871,0.743991,0.997871,0.756308,332.068794
9,0.997352,0.997352,0.783911,0.997352,0.663056,0.997352,0.666251,332.704665


In [20]:
results.to_csv(STATS_OUTPUT)